# Project 4: Decision Trees and Random Forest

### Name:
### Course Level:

**Introduction:**
* In this project, we explore the application of classification using: a) Decision Tree and b) Random Forest.

**Objectives:**
* The objective of this project is to develop software modules for classification of Titanic passengers via decision trees and random forests

## All Students

* The first problem we aim to us Information Gain to build a decision tree for classification.  The tree will complete when one of two conditions are met:

1. The maximum tree depth is reached.
2. The minimum number of samples is reached in a given leaf node.

**Problem A (70pts)**

1 (5pts). The first thing you'll need to do is read the Titatinic dataset from D2L (details about the dataset can be found [Here](https://www.kaggle.com/c/titanic/data?select=test.csv)).  Note that you will need both the train.csv (to build your tree) and test.csv to evaluate the classificaiton accuracy.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import random
import math
import matplotlib.pyplot as plt

# 1. Read in the data
def load_data():
    # Load training and test datasets
    train_data = pd.read_csv('train.csv')
    test_data = pd.read_csv('test.csv')
    
    # Basic preprocessing
    for i, df in enumerate([train_data, test_data]):
        # Fill missing age values with the median age - fixed to avoid warnings
        median_age = df['Age'].median()
        df['Age'] = df['Age'].fillna(median_age)
        
        # Fill missing embarked values with the most common value - fixed
        mode_embarked = df['Embarked'].mode()[0]
        df['Embarked'] = df['Embarked'].fillna(mode_embarked)
        
        # Convert categorical features to numerical
        df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
        
        # Creating age groups to simplify the tree
        df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 12, 18, 65, 100], 
                               labels=['Child', 'Teen', 'Adult', 'Elderly'])
        
        # Creating fare groups
        df['FareGroup'] = pd.cut(df['Fare'], bins=[0, 7.91, 14.45, 31, 513], 
                                labels=['Low', 'Medium', 'High', 'Very High'])
    
    # Select useful features
    features = ['Pclass', 'Sex', 'AgeGroup', 'SibSp', 'Parch', 'FareGroup', 'Embarked']
    
    X_train = train_data[features]
    y_train = train_data['Survived']
    
    X_test = test_data[features]
    y_test = test_data['Survived']
    
    return X_train, y_train, X_test, y_test

2 (20pts). Next, we need a couple helper functions to compute the conditional entropy and information gain.

**Recall: Entropy** 
$$
    H(X) = -\sum_{\textbf{x} \in X} p(\textbf{x}) \log_2( p(\textbf{x}) )
$$

**Recall: Conditional Entropy** 
$$
    H(Y|X=\textbf{x}) = -\sum_{\textbf{y} \in Y} p(\textbf{y}|\textbf{x}) \log_2 ( p(\textbf{y}|\textbf{x}) )
$$
where
$$
    p(\textbf{y}|\textbf{x}) = \frac{p(\textbf{y},\textbf{x})}{p(\textbf{x})}
$$
and
$$
    p(\textbf{x}) = \sum_{\textbf{y} \in Y} p(\textbf{y},\textbf{x})
$$

**Recall: Information Gain** 
$$
    IG(Y|X) = H(Y) - H(Y|X)
$$



In [2]:
# Function definitions here (you may want to modify the input parameters depending on your implimentation #
def Entropy(y):
    counter = Counter(y)
    total_samples = len(y)
    entropy = 0
    for count in counter.values():
        probability = count / total_samples
        entropy -= probability * math.log2(probability)
    
    return entropy

def C_Entropy(X, y, feature):
    # Get all unique values of the feature
    feature_values = set(X[feature])
    total_samples = len(y)
    conditional_entropy = 0
    
    for value in feature_values:
        # Filter samples where feature = value
        indices = X[feature] == value
        subset_y = y[indices]
        
        weight = len(subset_y) / total_samples
        
        # Add weighted entropy of this subset
        conditional_entropy += weight * Entropy(subset_y)
    
    return conditional_entropy

def IG(X, y, feature):
    original_entropy = Entropy(y)
    cond_entropy = C_Entropy(X, y, feature)
    information_gain = original_entropy - cond_entropy
    
    return information_gain

3 (30pts). Define a function to build the tree, recall there should be one of two exit criterion:

1. The maximum tree depth is reached
2. The minimum number of samples is reached in a given leaf node.


In [3]:
# Build Tree (can be recursive) #
def BuildTree(X, y, features, max_depth=3, min_samples=5, current_depth=0):
    # Convert Series to array for easier handling
    if isinstance(y, pd.Series):
        y = y.values
    
    # Check if stopping criteria are met
    if current_depth >= max_depth or len(y) <= min_samples or len(set(y)) == 1:
        # Return the most common class as leaf node
        most_common_class = Counter(y).most_common(1)[0][0]
        return {'type': 'leaf', 'class': most_common_class}
    
    # If no features left, return leaf node
    if not features:
        most_common_class = Counter(y).most_common(1)[0][0]
        return {'type': 'leaf', 'class': most_common_class}
    
    # Find the best feature to split on
    max_gain = -float('inf')
    best_feature = None
    
    for feature in features:
        gain = IG(X, y, feature)
        if gain > max_gain:
            max_gain = gain
            best_feature = feature
    
    # If no good split found, create leaf node
    if best_feature is None:
        most_common_class = Counter(y).most_common(1)[0][0]
        return {'type': 'leaf', 'class': most_common_class}
    
    # Create a decision node for the best feature
    tree = {'type': 'node', 'feature': best_feature, 'children': {}}
    
    # Get unique values of the best feature
    feature_values = set(X[best_feature])
    
    # Create subtrees for each value of the best feature
    for value in feature_values:
        # Filter samples where best_feature = value
        indices = X[best_feature] == value
        subset_X = X.loc[indices]
        subset_y = y[indices] if isinstance(y, np.ndarray) else y.loc[indices]
        
        # Skip if no samples with this value
        if len(subset_y) == 0:
            tree['children'][value] = {'type': 'leaf', 'class': Counter(y).most_common(1)[0][0]}
            continue
        
        # Recursively build subtree
        remaining_features = [f for f in features if f != best_feature]
        subtree = BuildTree(subset_X, subset_y, remaining_features, 
                          max_depth, min_samples, current_depth + 1)
        
        tree['children'][value] = subtree
    
    return tree

4 (15pts). Next write a function to perform the prediciton (classificaiton) of a new training sample:

In [4]:
# Function to predict #
def DT_predict(tree, sample):
    # If leaf node, return the class
    if tree['type'] == 'leaf':
        return tree['class']
    
    # Get the feature value from the sample
    feature = tree['feature']
    value = sample[feature]
    
    # If this value wasn't seen during training, return the most common class
    if value not in tree['children']:
        # Find any child and get its class (assuming all leaves have same class)
        for child in tree['children'].values():
            if child['type'] == 'leaf':
                return child['class']
        # If no leaf found, just return 0 as default
        return 0
    
    # Navigate to the next node based on the feature value
    return DT_predict(tree['children'][value], sample)

5 (5pts). Using the test set, evaluate the accuracy of your classification tree.

In [5]:
# Classification Accuracy #
def accuracy(X_test, y_test, tree):
    correct = 0
    total = len(y_test)
    
    for i in range(total):
        sample = X_test.iloc[i]
        prediction = DT_predict(tree, sample)
        if prediction == y_test.iloc[i]:
            correct += 1
    
    return correct / total

**Problem B (30pts)**
* In Problem A, you designed a function to build a decision tree.  In this problem, you will use that function to build several different trees via bagging (bootstrapping and aggregation).  The result will be a random forest.  You should **at minimum** have a forest of at least 5 trees. 

* Once the forest is built, similar to A.5, design a function for comparing the accuracy of a standard tree, vs. random forest on the testing dataset.



In [6]:
# Build Random Forest #
def bootstrap_sample(X, y):
    n_samples = len(X)
    indices = np.random.choice(range(n_samples), size=n_samples, replace=True)
    return X.iloc[indices].reset_index(drop=True), y.iloc[indices].reset_index(drop=True)

def build_random_forest(X, y, features, n_trees=5, max_depth=3, min_samples=5):
    forest = []
    
    for _ in range(n_trees):
        # Create bootstrap sample
        X_sample, y_sample = bootstrap_sample(X, y)
        
        # Select a random subset of features for this tree
        n_features = max(1, int(len(features) * 0.7))  # Use 70% of features
        sampled_features = random.sample(features, n_features)
        
        # Build a tree with this sample
        tree = BuildTree(X_sample, y_sample, sampled_features, max_depth, min_samples)
        
        # Add tree to the forest
        forest.append(tree)
    
    return forest

def rf_predict(forest, sample):
    predictions = [DT_predict(tree, sample) for tree in forest]
    return Counter(predictions).most_common(1)[0][0]

def rf_accuracy(X_test, y_test, forest):
    correct = 0
    total = len(y_test)
    
    for i in range(total):
        sample = X_test.iloc[i]
        prediction = rf_predict(forest, sample)
        if prediction == y_test.iloc[i]:
            correct += 1
    
    return correct / total

# Visualization function for comparing DT and RF performance
def plot_accuracy_comparison(dt_acc, rf_acc, n_trees):
    models = ['Decision Tree', f'Random Forest ({n_trees} trees)']
    accuracies = [dt_acc, rf_acc]
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(models, accuracies, color=['skyblue', 'lightgreen'])
    
    # Add accuracy values on top of bars
    for bar, acc in zip(bars, accuracies):
        plt.text(bar.get_x() + bar.get_width()/2, acc + 0.01, f'{acc:.4f}', 
                ha='center', va='bottom', fontweight='bold')
    
    plt.ylim(0, 1.0)
    plt.ylabel('Accuracy')
    plt.title('Titanic Classification: Decision Tree vs Random Forest')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Save the figure
    plt.savefig('dt_vs_rf_accuracy.png')
    plt.close()

In [7]:
def main():
    # Load and preprocess data
    X_train, y_train, X_test, y_test = load_data()
    
    # Get features
    features = X_train.columns.tolist()
    
    # Build a single decision tree
    decision_tree = BuildTree(X_train, y_train, features, max_depth=5, min_samples=10)
    
    # Calculate accuracy of the decision tree
    dt_acc = accuracy(X_test, y_test, decision_tree)
    print(f"Decision Tree Accuracy: {dt_acc:.4f}")
    
    # Build a random forest
    n_trees = 10  # Using 10 trees for better accuracy
    random_forest = build_random_forest(X_train, y_train, features, n_trees=n_trees, 
                                      max_depth=5, min_samples=10)
    
    # Calculate accuracy of the random forest
    rf_acc = rf_accuracy(X_test, y_test, random_forest)
    print(f"Random Forest Accuracy (with {n_trees} trees): {rf_acc:.4f}")
    
    # Compare the results
    print(f"Improvement with Random Forest: {(rf_acc - dt_acc) * 100:.2f}%")
    
    # Visualize the comparison
    plot_accuracy_comparison(dt_acc, rf_acc, n_trees)

if __name__ == "__main__":
    main()

Decision Tree Accuracy: 0.7799
Random Forest Accuracy (with 10 trees): 0.7632
Improvement with Random Forest: -1.67%
